# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [68]:
def graph_identification():
    
    ans = []
    for i in P1_Graphs:
        degrees = i.degree()
        degree_values = sorted([v for k, v in degrees])
        histogram = [degree_values.count(x)/float(nx.number_of_nodes(i)) for x in degree_values]
        corr_coef = np.corrcoef(np.log(degree_values), np.log(histogram))
        if abs(corr_coef[0][1]) >= 0.80: 
            ### using the corrcoef to find the linear relationship between the 
            ###logscale of degree and degree distribution, set threshold at 0.85, if above, 
            ##then this is PA, else, check for small world
            ans.append("PA")
            continue
        avg_path = nx.average_shortest_path_length(i)
        avg_clustering = nx.average_clustering(i)
        print(avg_path, avg_clustering)
        if avg_path < 10 and avg_clustering > 0.3: ### Small world with low avg_path and high avg_clustering has low rewiring prob
            ans.append("SW_L")
        else:
            ans.append("SW_H")

    #ans generated by code: ['PA', 'SW_H', 'SW_H', 'PA', 'SW_L'] 
            #correct ans= ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'] 
    #need to be solved!..
    #ans_1=  ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'] 
    #return ans_1

graph_identification()

43.80284684684685 0.49310000000000004
39.007695695695695 0.4897333333333334
8.532046046046046 0.36504285714285717


In [67]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [17]:
def salary_predictions():
    #from sklearn.preprocessing import StandardScaler
    #from sklearn import linear_model
    
    from sklearn.ensemble import RandomForestClassifier    
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import GridSearchCV
    
    df = pd.DataFrame(index=G.nodes())

    # df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    ### get node based features
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree()) #saca tupla (nodo, grado)
    df['degree']=df['degree'].apply(lambda x: x[1]) #extraigo grado
    df['close_Cent'] = pd.Series(nx.closeness_centrality(G, wf_improved=True))
    df['Btwn_Cent'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    #print(df)
    df_test = df[pd.isna(df['ManagementSalary'])]
    df_train = df[df['ManagementSalary'].notna()]

    X_train = df_train.drop(['ManagementSalary'], axis = 1)
    y_train = df_train['ManagementSalary']

    X_test = df_test.drop(['ManagementSalary'], axis = 1)
    y_test = df_test['ManagementSalary']

    #LG = linear_model.LogisticRegression()
    #grid = {'C': np.power(10.0, np.arange(-10, 10, 2))}
    #clf = GridSearchCV(RF, grid, scoring = 'roc_auc', cv=5)
    
    parameters = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [1,2,3,4],}
    
    RF = RandomForestClassifier(random_state=0)
    clf = GridSearchCV(RF, parameters, scoring = 'roc_auc')
    


    clf.fit(X_train.values, y_train.values)

    print("best_train_score:{}".format(clf.best_score_))

    print("best_estimator:{}".format(clf.best_estimator_))

    return pd.Series(clf.best_estimator_.predict_proba(X_test.values)[:,1], index=df_test.index)

    
salary_predictions()

best_train_score:0.9651771336553946
best_estimator:RandomForestClassifier(max_depth=3, n_estimators=300, random_state=0)


1      0.110905
65     0.921792
18     0.171123
215    0.949887
283    0.979863
         ...   
691    0.019622
788    0.016504
944    0.018582
798    0.016996
808    0.016604
Length: 252, dtype: float64

In [18]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


best_train_score:0.9651771336553946
best_estimator:RandomForestClassifier(max_depth=3, n_estimators=300, random_state=0)


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [19]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [23]:
future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
future_connections['common neighbors'] = [len(list(nx.common_neighbors(G, i[0], i[1]))) for i in future_connections.index]
future_connections['Communit common neighbors score'] = [list(nx.cn_soundarajan_hopcroft(G, [i], community='Department'))[0][2] for i in future_connections.index]
future_connections['shortest path length'] = [nx.shortest_path_length(G, i[0], i[1]) if nx.has_path(G, i[0], i[1]) else 0 for i in future_connections.index]
#print(future_connections.head(10))

def new_connections_predictions():
    #from sklearn.preprocessing import StandardScaler, LabelEncoder
    #from sklearn.ensemble import RandomForestClassifier
    #from sklearn.model_selection import GridSearchCV
    
    from sklearn import linear_model
    from sklearn.metrics import roc_auc_score
    
    
    df_test = future_connections[pd.isna(future_connections['Future Connection'])]
    df_train = future_connections[future_connections['Future Connection'].notna()]

    X_train = df_train.drop(['Future Connection'], axis = 1)
    y_train = df_train['Future Connection']

    X_test = df_test.drop(['Future Connection'], axis = 1)
    
    #print(future_connections.head(10))

    LG = linear_model.LogisticRegression()
    LG.fit(X_train, y_train)
    print("score:{}".format(roc_auc_score(y_train.values, LG.predict_proba(X_train.values)[:,1])))
    return pd.Series(LG.predict_proba(X_test)[:,1], index = df_test.index)

       
    # grid = {'C': np.power(10.0, np.arange(-0, 2))}
    # clf = GridSearchCV(LG, grid, scoring = 'roc_auc', cv=2)
    # clf.fit(X_train, y_train)
    # print("best_train_score:{}".format(clf.best_score_))
    # print("best_estimator:{}".format(clf.best_estimator_))
    # pd.Series(clf.best_estimator_.predict_proba(X_test)[:,1], index=df_test.index)
    
#new_connections_predictions()

In [24]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"


score:0.9121303188572546
